### Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

In [2]:
USE_GPU = True
num_class = 100
dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100


In [3]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image


### Neural Network

In [4]:
import torch.nn.functional as F  # useful stateless functions

In [73]:
class TestNet(nn.Module): # 1D Convolution
    def __init__(self, in_channel, num_classes):
        super(TestNet, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv1d(in_channel, 16, kernel_size=3, stride = 1, padding = 1),
            nn.BatchNorm1d(16),
            nn.ReLU()
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv1d(16, 32, kernel_size=3, stride = 1, padding = 1),
            nn.BatchNorm1d(32),
            nn.ReLU()
        )

        self.layer3 = nn.Sequential(
            nn.Conv1d(32, 64, kernel_size=3, stride = 1, padding = 1),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )
        
        self.lin = nn.Sequential(
            nn.Linear(16 * 1000, num_classes) # ? should be 64.. must look into further..
        )
    
    def forward(self, x):
        output = self.layer1(x)
        output = self.lin(flatten(output))
        return output

#### Testing

In [74]:
x = torch.zeros((64, 7, 1000), dtype=dtype) # batch, channels, duration

In [75]:
model = TestNet(in_channel=7, num_classes = 3)

In [76]:
model(x).shape # should be [64, 3]

torch.Size([64, 3])

### Dataloading and then testing:

In [77]:
def check_accuracy(loader, model):
    # if loader.dataset.train:
    #     print('Checking accuracy on validation set')
    # else:
    #     print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
    return acc

In [1]:
def training(model, optimizer, epochs=1):
    """
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: The accuracy of the model
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if (t + 1) % print_every == 0:
                print('Epoch %d, Iteration %d, loss = %.4f' % (e, t + 1, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()
    return check_accuracy(loader_val, model)